In [1]:
import os
import torch
import clip
from PIL import Image

from  torch.utils.data  import Dataset, DataLoader
import numpy as np
from sklearn.linear_model import LogisticRegression
from torch.utils.data import DataLoader
from tqdm import tqdm
from collections import OrderedDict

In [2]:
class PCam200(Dataset):
    def __init__(self, dataset_dir, transform=None):
        ext = ['.JPG', '.jpg', '.JPEG', '.png']
        self.transform = transform
        self.root_dir = dataset_dir
        self.normals = sorted( os.listdir(os.path.join(self.root_dir, 'normal')) )
        self.tumors = sorted( os.listdir(os.path.join(self.root_dir, 'tumor')) )
        try: 
            self.dirs.remove('thumbnail_position_map')
        except:
            print()
        self.data = []
        self.label = []
        idx = 0
        for d in self.normals:
            slide_dir = os.path.join(self.root_dir, 'normal', d)
            if os.path.isdir(slide_dir):
                imgs = sorted( os.listdir(slide_dir) )
                for img in imgs:
                    if img[img.find('.'): ] in ext:
                        self.data.append(os.path.join(slide_dir, img))
                        self.label.append(0)
                        idx += 1
        for d in self.tumors:
            slide_dir = os.path.join(self.root_dir, 'tumor', d)
            if os.path.isdir(slide_dir):
                imgs = sorted( os.listdir(slide_dir) )
                for img in imgs:
                    if img[img.find('.'): ] in ext:
                        self.data.append(os.path.join(slide_dir, img))
                        self.label.append(1)
                        idx += 1
    def __len__(self):
        return len(self.data)
    def __getitem__(self, idx):
        with open(self.data[idx] , 'rb') as f:
            img = Image.open(f).convert('RGB')
        return self.transform(img), torch.tensor(self.label[idx], dtype=torch.int64)


In [3]:
# load model
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load('ViT-B/32', device=device)


In [4]:
# zero-shot prediction
text_inputs = torch.cat([clip.tokenize(f"a photo of lymph node {c} tissue") for c in ['tumor', 'normal']]).to(device) 
for img in os.listdir("./images"):
    print(img)
    image_input = preprocess(Image.open(os.path.join("./images", img))).unsqueeze(0).to(device) 
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    print("  Tumor : {:.2f}".format( similarity[0][0].item() ))
    print("  Normal: {:.2f}".format( similarity[0][1].item() ))


normal021_0000026.jpg
  Tumor : 0.79
  Normal: 0.21
normal054_0000014.jpg
  Tumor : 0.74
  Normal: 0.26
tumor007_0000044.jpg
  Tumor : 0.69
  Normal: 0.31
tumor016_0000038.jpg
  Tumor : 0.71
  Normal: 0.29


In [6]:
# linear probe
def get_features(dataset_dir, transform=None):
    dataset = PCam200(dataset_dir, transform=transform)
    all_features = []
    all_labels = []
    
    with torch.no_grad():
        for images, labels in tqdm(DataLoader(dataset, batch_size=256)):
            features = model.encode_image(images.to(device))

            all_features.append(features)
            all_labels.append(labels)

    return torch.cat(all_features).cpu().numpy(), torch.cat(all_labels).cpu().numpy()

# Calculate the image features
train_features, train_labels = get_features("./PCam200/train", preprocess)
test_features, test_labels = get_features("./PCam200/test", preprocess)

# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

  0%|                                                                                          | 0/112 [00:00<?, ?it/s]

  0%|                                                                                           | 0/70 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [01:51<00:00,  1.59s/it]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy = 82.251


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    4.3s finished


In [7]:
# train epochs
epochs = 3
# batch size, reduce batch_size if out of memory error occurs
batch_size = 256

# Linear head transfer learning
new_model = torch.nn.Sequential( OrderedDict([
                                   
                                      ('clip',  model.visual),
                                      ('head',  torch.nn.Linear(model.visual.proj.size(1), 2, bias=True))
                                  ])
                               )
# initialize bias of added linear layer as zero
torch.nn.init.zeros_( new_model.head.bias.data )

# cast dtypes weight and bias of added linear layer to model dtype
new_model.head.weight.data = new_model[-1].weight.data.to(model.dtype)
new_model.head.bias.data   = new_model[-1].bias.data.to(model.dtype)
new_model.to(device)
    
# prepare dataset and dataloader
train_set = PCam200("./PCam200/train", transform=preprocess)
test_set  = PCam200("./PCam200/test", transform=preprocess)

train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_set,  batch_size=batch_size, shuffle=False)

# define optimizer and loss function
optimizer = torch.optim.SGD([{"params": new_model[-1].parameters(), "lr": 1e-3},  
                             {"params": new_model[0].parameters(), "lr": 1e-4}],  # smaller learning rate for backbone model
                              momentum=0.9)
criterion = torch.nn.CrossEntropyLoss()

train_iterator = tqdm( train_loader )
test_iterator  = tqdm( test_loader  )
# training
for epoch in range(epochs):
    print(f"{epoch} / {epochs}  epoch")
    for batch in train_iterator:
        img, label = batch[0].to(model.dtype).to(device), batch[1].to(device)
        new_model.train()
        
        out = new_model(img)
        loss = criterion(out, label)
        
        optimizer.zero_grad()
        loss.backward()      # calculate gradients
        optimizer.step()     # update parameters
        
# testing
total = len(test_set)
correct = 0.0
for batch in test_iterator:
    img, label = batch[0].to(model.dtype).to(device), batch[1].to(device)
    new_model.eval()

    with torch.no_grad():
        out = new_model(img)

        _, predicted = torch.max(out, 1)
        correct += (predicted == label).sum().item()

# Evaluate accuracy on test set
accuracy = (correct/total) * 100.
print(f"{epochs} epochs of fine-tuning yields,")
print(f"Accuracy = {accuracy:.3f}")

  0%|                                                                                          | 0/112 [00:00<?, ?it/s]


  0%|                                                                                           | 0/70 [00:00<?, ?it/s]

0 / 3  epoch


100%|████████████████████████████████████████████████████████████████████████████████| 112/112 [03:03<00:00,  1.64s/it]


1 / 3  epoch
2 / 3  epoch



 94%|█████████████████████████████████████████████████████████████████████████████▎    | 66/70 [11:00<00:06,  1.71s/it]


100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [11:05<00:00,  9.51s/it]

3 epochs of fine-tuning yields,
Accuracy = 89.567


In [10]:
# zero-shot prediction after fine-tuning
text_inputs = torch.cat([clip.tokenize(f"a photo of lymph node {c} tissue") for c in ['tumor', 'normal']]).to(device) 
for img in os.listdir("./images"):
    print(img)
    image_input = preprocess(Image.open(os.path.join("./images", img))).unsqueeze(0).to(device) 
    with torch.no_grad():
        image_features = model.encode_image(image_input)
        text_features = model.encode_text(text_inputs)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)
    similarity = (100.0 * image_features @ text_features.T).softmax(dim=-1)
    print("  Tumor : {:.2f}".format( similarity[0][0].item() ))
    print("  Normal: {:.2f}".format( similarity[0][1].item() ))

normal021_0000026.jpg
  Tumor : 0.36
  Normal: 0.64
normal054_0000014.jpg
  Tumor : 0.23
  Normal: 0.77
tumor007_0000044.jpg
  Tumor : 0.67
  Normal: 0.33
tumor016_0000038.jpg
  Tumor : 0.50
  Normal: 0.50


In [9]:
# linear probe after fine-tuning

# Calculate the image features
train_features, train_labels = get_features("./PCam200/train", preprocess)
test_features, test_labels = get_features("./PCam200/test", preprocess)

# Perform logistic regression
classifier = LogisticRegression(random_state=0, C=0.316, max_iter=1000, verbose=1)
classifier.fit(train_features, train_labels)

# Evaluate using the logistic regression classifier
predictions = classifier.predict(test_features)
accuracy = np.mean((test_labels == predictions).astype(np.float)) * 100.
print(f"Accuracy = {accuracy:.3f}")

  0%|                                                                                           | 0/70 [00:00<?, ?it/s]

100%|██████████████████████████████████████████████████████████████████████████████████| 70/70 [01:51<00:00,  1.59s/it]
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


Accuracy = 89.521


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    2.4s finished
